In [37]:
from sudoku import Sudoku

In [38]:
# Unsolved 9x9 sudoku with 80 clues
#s = Sudoku("327456891918372546465981732532864179791523684846719253653148927279630418184297365")

# Unsolved 9x9 sudoku with 75 clues
#s = Sudoku("192700658437865912560291347749358126053612479216947583625179830971483265084526791")

# Unsolved 9x9 sudoku with 75 clues
#s = Sudoku("192700658437865912560291347749358126053612479216947583625179830971483265084526791")

# Unsolved 9x9 sudoku with 17 clues
s = Sudoku("600000900020000000000000000000000059100080000000300200005060407890040000004000000")

# Solved 9x9 sudoku with 17 clues
#s = Sudoku("683417925529638174471592386367124859142985763958376241215869437896743512734251698")


In [39]:
def tuples_liste_taille_1(dictionnaire):
    resultats = [cle for cle, valeur in dictionnaire.items() if len(valeur) == 1]
    return resultats

In [40]:
from collections import Counter
from random import choices, choice

restart = 0

solutionFound = False

moves = s.getPossibleMoves()

while not solutionFound:
  # If there is a cell with only 1 possibility we chose it immediatly
  if len(tuples_liste_taille_1(moves)) > 0:
    coords = tuples_liste_taille_1(moves)[0]
    s.doMove(coords, moves[coords][0])
  # Else, we rank our cells by number of possiblities and assign it a probability to be choosed
  else:
    movesBis = []
    for move in moves:
      movesBis.append(((move),len(moves[move])))
    
    uniquePossibilities = list(set([x[1] for x in movesBis]))
    print(moves)
    probabilites = [1 / valeur for valeur in uniquePossibilities]
    probabilites_normalized = [prob / sum(probabilites) for prob in probabilites]
    occurencesOfPossibilities = dict(Counter([x[1] for x in movesBis]))

    probas = {}
    for i in range(len(probabilites_normalized)):
      probas[uniquePossibilities[i]] = probabilites_normalized[i] / occurencesOfPossibilities[uniquePossibilities[i]]

    #probas = list(probas.items())

    movesBis = [ (x[0], probas[x[1]]) for x in movesBis]

    cellChosen = choices([x[0] for x in movesBis], weights=[x[1] for x in movesBis])[0]

    numberChosen = choice(moves[cellChosen])
    s.doMove(cellChosen, numberChosen)

  if s.isSolved():
    break;
  
  moves = s.getPossibleMoves()
  
  if not s.isThereAMove():
    s = Sudoku("600000900020000000000000000000000059100080000000300200005060407890040000004000000")
    restart+=1

  for m in moves:
    # There is a cell we want to filled but has no possible value (because of constraintes)
    if len(moves[m]) == 0:
      print(m)
      print("Locked")
      s = Sudoku("600000900020000000000000000000000059100080000000300200005060407890040000004000000")
      restart+=1
      break

  moves = s.getPossibleMoves()

  s.printBoard()
  print("\n##############################################\n")

print("nb restart :", restart)

{(0, 1): [1, 3, 4, 5, 7, 8], (0, 2): [1, 3, 7, 8], (0, 3): [1, 2, 4, 5, 7, 8], (0, 4): [1, 2, 3, 5, 7], (0, 5): [1, 2, 3, 4, 5, 7, 8], (0, 7): [1, 2, 3, 4, 7, 8], (0, 8): [1, 2, 3, 4, 5, 8], (1, 0): [3, 4, 5, 7, 9], (1, 2): [1, 3, 7, 8, 9], (1, 3): [1, 4, 5, 6, 7, 8, 9], (1, 4): [1, 3, 5, 7, 9], (1, 5): [1, 3, 4, 5, 6, 7, 8, 9], (1, 6): [1, 3, 5, 6, 7, 8], (1, 7): [1, 3, 4, 6, 7, 8], (1, 8): [1, 3, 4, 5, 6, 8], (2, 0): [3, 4, 5, 7, 9], (2, 1): [1, 3, 4, 5, 7, 8], (2, 2): [1, 3, 7, 8, 9], (2, 3): [1, 2, 4, 5, 6, 7, 8, 9], (2, 4): [1, 2, 3, 5, 7, 9], (2, 5): [1, 2, 3, 4, 5, 6, 7, 8, 9], (2, 6): [1, 3, 5, 6, 7, 8], (2, 7): [1, 2, 3, 4, 6, 7, 8], (2, 8): [1, 2, 3, 4, 5, 6, 8], (3, 0): [2, 3, 4, 7], (3, 1): [3, 4, 6, 7, 8], (3, 2): [2, 3, 6, 7, 8], (3, 3): [1, 2, 4, 6, 7], (3, 4): [1, 2, 7], (3, 5): [1, 2, 4, 6, 7], (3, 6): [1, 3, 6, 7, 8], (4, 1): [3, 4, 5, 6, 7], (4, 2): [2, 3, 6, 7, 9], (4, 3): [2, 4, 5, 6, 7, 9], (4, 5): [2, 4, 5, 6, 7, 9], (4, 6): [3, 6, 7], (4, 7): [3, 4, 6, 7], (4, 8